# Various utilities 

This Notebook presents some usefull utilities included in `pyposeidon`. 


- Requires:

    - The *./test/global/results.nc* file from the [Global Storm Surge](GSS.ipynb) Notebook.

    - The data folder. See [README](README.md) in this folder.

In [ ]:
#to use the full width of the browser window
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Not neccessarily needed. Just to check version. Must be >= 0.6
import pyposeidon
pyposeidon.__version__

In [ ]:
import pyposeidon.utils.pplot # initialize matplolib accessor

In [ ]:
#optional
#pyposeidon.utils.pplot.__init__(dark_background=True) # set plt style for pplot graphics below

In [ ]:
import pyposeidon.utils.mplot # initialize pyviz accessor

In [ ]:
import xarray as xr

In [ ]:
import numpy as np

## 2D Visualization of a global mesh

When working with a global mesh the 2D representation of the data is problematic.

In [ ]:
r = xr.open_dataset('./test/global/results.nc')

In [ ]:
r

We can visualize in 3D correctly with

In [ ]:
r.mplot.mesh(dim='3D')

But in 2D we get artifacts due to the cross over elements...

In [ ]:
r.pplot.mesh()

However, using the corresponding utility from `pyposeidon` we can create a seam on the international time line mitigating the problem.

In [ ]:
from pyposeidon.utils.seam import get_seam

In [ ]:
#original mesh topology
x = r.SCHISM_hgrid_node_x[:].values
y = r.SCHISM_hgrid_node_y[:].values
tri3 = r.SCHISM_hgrid_face_nodes.values[:,:3].astype(int)

In [ ]:
x.shape, y.shape, tri3.shape

In [ ]:
xn,yn,tri3n= get_seam(x,y,None,tri3) # get seamed arrays 

In [ ]:
#These arrays can be saved for future use (see below)
np.save("./test/global/to2d", [xn, yn, tri3n])
#load if needed
#[xn, yn, tri3n] = np.load("./test/global/to2d.npy",allow_pickle=True)

In [ ]:
# we see that the number of elements and nodes increased due to the split
xn.shape, yn.shape, tri3n.shape

In [ ]:
r.pplot.mesh(x=xn,y=yn,tes=tri3n)

The creation of the seam can best visualized in 3D

In [ ]:
r.mplot.mesh(x=xn,y=yn,tes=tri3n,dim='3D')

## Get variables on the new 2D mesh

Having the new 2D topology we can compute the corresponding values for any variable in a dataset.

Here we use the data from the global run.

In [ ]:
from pyposeidon.utils.seam import to_2d

Specifying the variable we can then extract the new dataset e.g. for "depth"

In [ ]:
bathymetry = to_2d(r,var='depth', mesh=[xn,yn,tri3n])

In [ ]:
bathymetry.mplot.contourf(var='depth', dim='3D') # plot in 3D

.. or  in 2D ..

In [ ]:
bathymetry.pplot.contourf(var='depth')

Likewise for the time variables...

In [ ]:
sl = to_2d(r,var='elev', mesh=[xn,yn,tri3n])

In [ ]:
# animate in 2D
sl.pplot.frames(var='elev',title='SSH')

In [ ]:
#this dataset can be saved to a new netcdf file as usual
sl.to_netcdf('./test/global/elev.nc')

In [ ]:
# We can plot the elevation data for the last timestep in 3D as
sl.mplot.contourf(var='elev', it=-1, dim='3D')